### 1. Import Libraries

In [228]:
import numpy as np
import pandas as pd
import seaborn as sns
import csv
from datetime import datetime as dt
import pandas_profiling
import matplotlib.pyplot as plt



### 2. Loading Data

In [8]:
#abrindo arquivo csv
data=pd.read_csv('/home/isisaraujo/Repos/Projeto_piloto.PMSE/1Cipm/datasets/1trimestre2020.csv')
data.head()

,Data de Criação,Hora de Criação,Ocorrência,Endereço,Bairro,Município,Cód. Tipo,Descrição,Cód. Sub Tipo,Desc. Sub Tipo,...,Despachado,Tempo Despacho,Chegada Local,Tempo Deslocamento,Fechado,Placa,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,01-01-2020,01:16:04,M2280478,"RUA FR SANTA CECILIA,",.A DEFINIR,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,E08420.2,EM VEICULO,...,-,::,-,-,01-01-2020 00:17:35,NaN,NaN,NaN,NaN,NaN
1,01-01-2020,00:25:57,M2280492,"RUA CEMITERIO, :NUMERAL INFORMADO 0",.A DEFINIR,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,E08420.2,EM VEICULO,...,-,::,-,-,01-01-2020 00:57:38,NaN,NaN,NaN,NaN,NaN
2,01-01-2020,00:28:12,M2280497,"RUA B ROSA ELZE, 436",.CJ ROSA ELZE,SÃO CRISTOVÃO,A02000,AGRESSAO VERBAL,A02000.1,DENTRO DE RESIDENCIA,...,01-01-2020 00:31:33,00: 03: 21,-,-,01-01-2020 01:11:21,NaN,NaN,NaN,NaN,NaN
3,01-01-2020,01:56:06,M2280531,"RUA NOVENTA E TRES CJ BRG EDUARDO GOMES, 8 :NU...",.CJ BRG EDUARDO GOMES,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,E08420.5,EM VIA PUBLICA,...,-,::,-,-,01-01-2020 00:57:39,NaN,NaN,NaN,NaN,NaN
4,01-01-2020,01:01:00,M2280539,"RUA JOAO PAULO DOS PASSOS, 2 :NUMERAL INFORMADO 0",.LTM ROSA MARIA,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,E08420.2,EM VEICULO,...,-,::,-,-,01-01-2020 01:41:17,NaN,NaN,NaN,NaN,NaN


In [9]:
#listando locunas
data.columns

Index(['Data de Criação', 'Hora de Criação', 'Ocorrência', 'Endereço',
       'Bairro', 'Município', 'Cód. Tipo', 'Descrição', 'Cód. Sub Tipo',
       'Desc. Sub Tipo', 'Sit. Encontrada', 'Batalhão', 'COMPANHIA', 'Viatura',
       'Cód. Final.', 'Descrição Finalização', 'Obs. Finalização',
       'Solicitante', 'Telefone', 'XCORD', 'YCORD', 'Aberto', 'Despachado',
       'Tempo Despacho', 'Chegada Local', 'Tempo Deslocamento', 'Fechado',
       'Placa', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31'],
      dtype='object')

###  3. Tratando os dados para extrair informações

In [10]:
#excluindo colunas
data=data.drop(['Hora de Criação', 'Ocorrência', 'Endereço',
       'Bairro', 'Cód. Sub Tipo',
       'Desc. Sub Tipo', 'Sit. Encontrada', 'Batalhão', 'COMPANHIA', 'Viatura',
       'Cód. Final.', 'Descrição Finalização', 'Obs. Finalização',
       'Solicitante', 'Telefone', 'XCORD', 'YCORD', 'Aberto', 'Despachado',
       'Tempo Despacho', 'Chegada Local', 'Tempo Deslocamento', 'Fechado',
       'Placa','Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31'], axis=1)

In [11]:
data.head()

,Data de Criação,Município,Cód. Tipo,Descrição
0,01-01-2020,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS
1,01-01-2020,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS
2,01-01-2020,SÃO CRISTOVÃO,A02000,AGRESSAO VERBAL
3,01-01-2020,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS
4,01-01-2020,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS


   ### 3.1 Achando os Dados nulus do Dataframe

In [12]:
#Achar dados nulos em um dataframe
data.isnull().sum().sort_values(ascending=False)[:10]

Município          4
Descrição          0
Cód. Tipo          0
Data de Criação    0
dtype: int64

In [13]:

# identificar o valor mais frequente na coluna município para ser substituído
print(data.Município.value_counts())
Município_freq = data.Município.value_counts()[0]

SÃO CRISTOVÃO        2986
ITAPORANGA DAJUDA     710
Name: Município, dtype: int64


In [14]:
#preencher valores faltando pelo pelo valor que mais aparece
data.Município.fillna(Município_freq, inplace=True)

In [15]:
# verificar se há valor ausente
data.Município.isnull().sum()

0

### 3.2 Incluir Coluna para posterior manipulação e tipos de variáveis


In [17]:
#verificando os tipos de variáveis
data.dtypes 

Data de Criação    object
Município          object
Cód. Tipo          object
Descrição          object
dtype: object

In [18]:
# Garantir que o formato date é um datetime(formato .br)
data['Data de Criação'] = pd.to_datetime( data['Data de Criação'], format='%d-%m-%Y' )

In [19]:
#Confirmando datetime
data.dtypes 

Data de Criação    datetime64[ns]
Município                  object
Cód. Tipo                  object
Descrição                  object
dtype: object

In [141]:
#Criando colunas de acordo com as datas e filtrando por meses
data['Data de Criação']=pd.to_datetime(data['Data de Criação'])
data['month']=data['Data de Criação'].dt.month
data.head()

,Data de Criação,Município,Cód. Tipo,Descrição,month
0,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1
1,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1
2,2020-01-01,SÃO CRISTOVÃO,A02000,AGRESSAO VERBAL,1
3,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1
4,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1


In [92]:
#renomeando as colunas e passando a trabalhar com DF1
df=data.rename(columns={'Data de Criação':'Data', 'month':'Meses'})
df1=df.copy()


In [139]:
df1

,Data,Município,Cód. Tipo,Descrição,Meses
0,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1
1,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1
2,2020-01-01,SÃO CRISTOVÃO,A02000,AGRESSAO VERBAL,1
3,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1
4,2020-01-01,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,1
...,...,...,...,...,...
3695,2020-03-31,SÃO CRISTOVÃO,A02000,AGRESSAO VERBAL,3
3696,2020-03-31,SÃO CRISTOVÃO,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,3
3697,2020-03-31,ITAPORANGA DAJUDA,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,3
3698,2020-03-31,ITAPORANGA DAJUDA,E08420,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,3


In [94]:

df1.value_counts(df1['Município'])

Município
SÃO CRISTOVÃO        2986
ITAPORANGA DAJUDA     710
2986                    4
dtype: int64

In [95]:
df1.value_counts(df1['Cód. Tipo'])[0:5]

Cód. Tipo
E08420    2018
B01147     251
B08210     250
G01330     248
A05000     147
dtype: int64

In [96]:
df1.value_counts(df1['Meses'])

Meses
3    1473
2    1189
1    1038
dtype: int64

### 4. Fazendo as Estatísticas com os dados Limpos

### 4.1 Cidade de São Cristovão

In [109]:
# Total das ocorrencias atendidass por cidade
# 1. Filtrar os dados da coluna municipio
Tot_sc = df1[df1['Município'] == 'SÃO CRISTOVÃO'].shape[0]
print( '\n O total de Ocorrências no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: {}'.format( Tot_sc ) )

#Valor total dividido por 3(( arredondarrr))
mes_sc= Tot_sc/3
print( '\n O total de Ocorrências no por Mês no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: {}'.format(mes_sc ) )

#Média diária ( arredondarrr)
md_sc= Tot_sc/90
print( '\n O total de Ocorrências atendidas por dia no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: {}'.format(md_sc ) )

#jan_sc=df1[(df1['Município'] == 'SÃO CRISTOVÃO')&(df1['Meses']=='1')].shape[0]
#print( '\n O total de Ocorrências atendidas no mês de Janeiro no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: {}'.format(jan_sc))


 O total de Ocorrências no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: 2986

 O total de Ocorrências no por Mês no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: 995.3333333333334

 O total de Ocorrências atendidas por dia no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: 33.17777777777778

 O total de Ocorrências atendidas no mês de Janeiro no 1º Trimestre de 2020 na cidade de SÃO CRISTOVÃO foi de: 0


### 4.2 Cidade de Itaporanga DAJUDA



In [98]:
# Total das ocorrencias atendidass por cidade
# 1. Filtrar os dados da coluna municipio
Tot_ip = df1[df1['Município'] == 'ITAPORANGA DAJUDA'].shape[0]
print( '\n O total de Ocorrências no 1º Trimestre de 2020 na cidade de ITAPORANGA DAJUDA foi de: {}'.format( Tot_ip ) )

#Valor total dividido por 3(( arredondarrr))
mes_ip= Tot_ip/3
print( '\n O total de Ocorrências no por Mês no 1º Trimestre de 2020 na cidade de ITAPORANGA DAJUDA foi de: {}'.format(mes_ip ) )

#Média diária ( arredondarrr)
md_ip= Tot_ip/90
print( '\n O total de Ocorrências atendidas por dia no 1º Trimestre de 2020 na cidade de ITAPORANGA DAJUDA foi de: {}'.format(md_ip ) )


 O total de Ocorrências no 1º Trimestre de 2020 na cidade de ITAPORANGA DAJUDA foi de: 710

 O total de Ocorrências no por Mês no 1º Trimestre de 2020 na cidade de ITAPORANGA DAJUDA foi de: 236.66666666666666

 O total de Ocorrências atendidas por dia no 1º Trimestre de 2020 na cidade de ITAPORANGA DAJUDA foi de: 7.888888888888889


### 5. Filtrando por Tipos de Ocorrẽncias

### 5.1 Cidade de São Cristovão

In [216]:
# filtrando por tipo de Ocorrências 
df2=df1[df1.Município=='SÃO CRISTOVÃO'].groupby('Descrição',sort=False).sum()[0:5].reset_index()
df2=df2.rename(columns={'Meses':'Total_Ocorrência'})
df3=df2.copy()
df3

,Descrição,Total_Ocorrência
0,PERTURBACAO DO TRABALHO OU DO SOSSEGO ALHEIOS,3270
1,AGRESSAO VERBAL,114
2,VIAS DE FATO - AGRESSAO,429
3,LESAO CORPORAL,111
4,TENTATIVA DE HOMICIDIO,63


In [ ]:
#Plotando Gráfico

### 5.2 Cidade de Itaporanga